# LLM "No Hint" Migration 
This notebook demonstrates how an LLM (using Kai's ModelProvider) performs when presented with a Spring Security migration issue without a hint.

Installing pre requisites and configuring Kai with `Llama-3.1-8B-Instruct`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [9]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 

# Initialize the model provider using Llama-3.1-8B-Instruc via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "meta-llama/Llama-3.1-8B-Instruct",
        "base_url": "https://llama-3-1-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1",
        "api_key": os.getenv("OPENAI_API_KEY"),},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [10]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [11]:
message=""""\
  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
    """

In [12]:
prompt = f"""
The following java code needs to be migrated from springboot2 to springboot3. 

```java
{before_code}
```
The issue has been identified by an analysis rule

{message}

Can you provide a refactored version of this code that is compatible with springboot3?
Also, create an hint based on your suggestions.

"""

In [13]:

response = await rendered_llm_call(prompt)

Here's the refactored version of the code that is compatible with Spring Boot 3:

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.core.userdetails.UserDetails;
import org.springframework.security.core.userdetails.UsernameNotFoundException;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

import java.util.Collections;

@Configuration
@EnableWebSecurity
public class SecurityConfig {
    @Bean
    public UserDetailsService userDetailsService() {
        return username -> {
            if (username.equals("user")) {
                return new User("user", passwordEncoder().encode("password"), Collections.singleton("USER"));
            } else {
                throw new UsernameNotFoundException("User not found");
            }
        };
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public SecurityConfigurerAdapter<HttpSecurity, HttpSecurity> securityConfigurerAdapter() {
        return http -> {
            http.authorizeRequests()
                    .antMatchers("/").permitAll()
                    .anyRequest().authenticated();
        };
    }
}
```

However, the above code still uses the `configure(AuthenticationManagerBuilder)` method which is discouraged in Spring Security 5.7.

Here's a better refactored version of the code that uses a `UserDetailsService` and `PasswordEncoder` bean:

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.core.userdetails.UserDetails;
import org.springframework.security.core.userdetails.UsernameNotFoundException;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

import java.util.Collections;

@Configuration
@EnableWebSecurity
public class SecurityConfig {
    @Bean
    public UserDetailsService userDetailsService() {
        return username -> {
            if (username.equals("user")) {
                return new User("user", passwordEncoder().encode("password"), Collections.singleton("USER"));
            } else {
                throw new UsernameNotFoundException("User not found");
            }
        };
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public SecurityConfigurerAdapter<HttpSecurity, HttpSecurity> securityConfigurerAdapter() {
        return http -> {
            http.authorizeRequests()
                    .antMatchers("/").permitAll()
                    .anyRequest().authenticated();
        };
    }
}
```

**Hint:** 

*   To fix the issue, you need to define a `UserDetailsService` and `PasswordEncoder` bean instead of overriding the `configure(AuthenticationManagerBuilder)` method.
*   The `UserDetailsService` bean should return a `UserDetails` object for the given username.
*   The `PasswordEncoder` bean should be used to encode the password.
*   You can use the `BCryptPasswordEncoder` class to encode the password.
*   You can use the `User` class from the `org.springframework.security.core.userdetails` package to create a `UserDetails` object.
*   You can use the `Collections.singleton` method to create a set of roles for the user.

**Best Practice:**

*   Instead of hardcoding the username and password, consider using a database or an external authentication service to store user credentials.
*   Consider using a more secure password encoder like `Argon2PasswordEncoder` or `PBKDF2PasswordEncoder` instead of `BCryptPasswordEncoder`.
*   Consider using a more secure way to handle user authentication, such as using a token-based authentication system.

### Evaluation

| Criteria                                                                 | Result         |
|--------------------------------------------------------------------------|----------------|
| Removed deprecated `configure(AuthenticationManagerBuilder)`            |  No           |
| Replaced with `UserDetailsService` and `PasswordEncoder` bean           | Partially Yes |
| Removed `WebSecurityConfigurerAdapter`                                   |  No           |
| The output compiles                                                      | <todo>          |
| Preserves behavior (in-memory auth, user with role)                      | No           |

